In [1]:
import pandas as pd
import torch 
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
from datetime import datetime as dt
import import_ipynb
from sari import SARIsent
from datasets import load_dataset, load_metric
from rouge_score import rouge_scorer
import numpy as  np
import optuna
import pickle

C:\Users\nkhozin\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing Jupyter notebook from sari.ipynb


In [32]:
model_name = "rut5_v7"
model = T5ForConditionalGeneration.from_pretrained(model_name).cuda()
tokenizer = T5Tokenizer.from_pretrained(model_name)
device='cuda'

In [33]:
val = pd.read_csv('data\\eval.csv', index_col=0)
val = val.sample(1000)

In [34]:
def objective(trial):    
    repetition_penalty = trial.suggest_float("repetition_penalty", 0, 1)
    num_beams = trial.suggest_categorical("num_beams", [2, 3, 4, 5])
    max_length = trial.suggest_float("max_length", 0.4, 1)
    length_penalty = trial.suggest_float("length_penalty", 0.2, 2.0)
    no_repeat_ngram_size = trial.suggest_int("no_repeat_ngram_size", 0, 5)
    early_stopping = trial.suggest_categorical("early_stopping", [True, False])
    
    def generate_t5(model, tokenizer, text):
        inputs = tokenizer(text, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            hypotheses = model.generate(inputs["input_ids"],
                                        num_beams=int(num_beams),
                                        length_penalty=length_penalty,
                                        no_repeat_ngram_size=int(no_repeat_ngram_size),
                                        early_stopping=early_stopping,
                                        max_length=int(len(text)*max_length)
                                       )
        return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

    val["pred"] = val.source.apply(lambda value: generate_t5(model, tokenizer, value))
    val["SARI"] = val.apply(lambda row: SARIsent(row['source'], row[f"pred"] ,row['target']), axis=1)*100
    print(val["SARI"].mean())
    return val["SARI"].mean()

In [36]:
study = optuna.create_study(direction='maximize')

[I 2023-05-29 00:07:47,226] A new study created in memory with name: no-name-f69f60ba-6b7e-44e1-ae41-c47a55d20618


In [37]:
%%time

study.optimize(objective, n_trials=75)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
[I 2023-05-29 00:12:53,110] Trial 0 finished with value: 33.38933977837658 and parameters: {'repetition_penalty': 0.8262450281160829, 'num_beams': 4, 'max_length': 0.8664604437707002, 'length_penalty': 1.7127475529047291, 'no_repeat_ngram_size': 3, 'early_stopping': True}. Best is trial 0 with value: 33.38933977837658.


33.38933977837658


[I 2023-05-29 00:17:45,266] Trial 1 finished with value: 33.4004049550285 and parameters: {'repetition_penalty': 0.3272026075593726, 'num_beams': 2, 'max_length': 0.504418909238358, 'length_penalty': 0.4200193732919183, 'no_repeat_ngram_size': 3, 'early_stopping': True}. Best is trial 1 with value: 33.4004049550285.


33.4004049550285


[I 2023-05-29 00:23:33,955] Trial 2 finished with value: 33.3815475258723 and parameters: {'repetition_penalty': 0.21327657559037239, 'num_beams': 3, 'max_length': 0.8582372561110928, 'length_penalty': 1.5183445075545265, 'no_repeat_ngram_size': 5, 'early_stopping': False}. Best is trial 1 with value: 33.4004049550285.


...

33.43583173366925


[I 2023-05-29 06:34:08,289] Trial 73 finished with value: 33.435481083275164 and parameters: {'repetition_penalty': 0.06232659003915873, 'num_beams': 4, 'max_length': 0.43900018373343974, 'length_penalty': 1.9432894331972088, 'no_repeat_ngram_size': 1, 'early_stopping': False}. Best is trial 61 with value: 33.436832361267065.


33.435481083275164


[I 2023-05-29 06:38:27,237] Trial 74 finished with value: 33.43108935710625 and parameters: {'repetition_penalty': 0.09336284253593126, 'num_beams': 2, 'max_length': 0.41415450278327237, 'length_penalty': 1.8900143448211226, 'no_repeat_ngram_size': 1, 'early_stopping': False}. Best is trial 61 with value: 33.436832361267065.


33.43108935710625
CPU times: total: 6h 29min 48s
Wall time: 6h 30min 39s


In [38]:
# Получение результатов
best_trial = study.best_trial

print("Best trial:")
print("Value: ", best_trial.value)
print("Params: ")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))

Best trial:
Value:  33.436832361267065
Params: 
repetition_penalty: 0.03657671556007456
num_beams: 4
max_length: 0.4193862846988271
length_penalty: 1.9554354329528894
no_repeat_ngram_size: 1
early_stopping: False


In [39]:
filehandler = open("optuna_predict_trials.obj","wb")
pickle.dump(study.get_trials(), filehandler)
filehandler.close()

In [41]:
with open('optuna_predict_trials.obj', 'rb') as f:
    obj = pickle.load(f)

In [42]:
tb = []

for i in range(len(obj)):
    trial = obj[i]   
    params = {}
    params['iter'] = trial.number
    params.update(trial.params)
    params['loss'] = trial.values[0]
    tb.append(params)

In [46]:
pd.DataFrame(tb).sort_values(by='loss', ascending=False).head(20)

,iter,repetition_penalty,num_beams,max_length,length_penalty,no_repeat_ngram_size,early_stopping,loss
61,61,0.036577,4,0.419386,1.955435,1,False,33.436832
64,64,0.037925,4,0.401239,1.998846,1,False,33.436202
62,62,0.046347,4,0.403628,1.996843,1,False,33.436191
72,72,0.046145,4,0.463121,1.957284,1,False,33.435832
51,51,0.117394,4,0.434174,1.947636,1,False,33.435636
63,63,0.038864,4,0.401139,1.987646,1,False,33.435633
60,60,0.071181,4,0.417438,1.947146,1,False,33.435579
53,53,0.006374,4,0.431077,1.939989,1,False,33.435574
73,73,0.062327,4,0.439000,1.943289,1,False,33.435481
71,71,0.028703,4,0.460353,1.991059,1,False,33.435477
